In [ ]:
from configparser import ConfigParser
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
from numpy import matrix
import pickle
from sklearn.metrics import mean_squared_error
from get_timbre import get_timbre
from evaluation import evaluate_data
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import math
pd.options.mode.chained_assignment = None  # default='warn'

parser = ConfigParser()
parser.read('./spotify_credentials.cfg')

SPOTIPY_CLIENT_ID = parser.get('spotify', 'SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = parser.get('spotify', 'SPOTIPY_CLIENT_SECRET')

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

user_id = 'czdoifmfngjhvoetavlok9dg5'

In [53]:
# playlist_ids = ['5odVaQ10ISGhvuakjVGmxp', '0mCV9EHiVXEsbw307lBF25', '4VGvI1dODxGyBop5jFhgfN',
#                 '5pVRL48wvxNRL8lWkxdVCX', '5Kq61WYIbYGLB6dvuTKzZV', '3MO4sQXwvB13TAdiSoG9AR',
#                 '64iSau8h74DkOUG8JVY3W2', '5ZWw4qko5ccxnEIA8blVSq', '35iVxmwsHyJVSdvRasuK8V',
#                 '06LB9aYKvibLEXb4Qf3gHM', '2dfxSJXVFYpZi2YOjgbwRX', '6FwDZbpBnGXBOJ1UwW9lKb',
#                 '44vN1eprBxUdirtb9pCOge', '64mSbvFQYIW547z4zWrEQK', '3VBWC197hTEmayqbDTmvp4',
#                 '2Am8cD0icf62d4dL7NrDTx', '0fjjY976NVJ03tPhxjamcX', '0vzdOGXJBf6SBFeYbwfpnL',
#                 '5JufAIzUR25lgE8zrums4k', '2ppGD3RwccxuPSJZiyZrCm', '4LBHOUvGlk4RtaMbfB5huK',
#                 '1VJZKBsulYCry6QpSoW5mG', '0dnsYgOOtfsOg78qLrZEAv', '5KtPLkVfM18fVQ4puapK98',
#                 '51eXnQHgNcFw7xGllU5kCf', '23x3CI4Cncf6BpSnIeZlF7', '1rDhjtHJkAIlUWgJlxQSRF',
#                 '5cS1vW9gzr2lLFZ0XpkmfD', '2xxXRaeMZSoAiUslTqbB7S', '0Gx1JfJVTkWSfCcTkMgJiA',
#                 ]
# random_y = ''

playlist_ids = ['5odVaQ10ISGhvuakjVGmxp']
random_y = ''

# playlist_ids = ['a','b','c','d']
# random_y = 'random_'

columns_to_remove = ['id', 'track_title', 'album_title', 'album_artist',
                    'track_number', 'total_tracks', 'key', 'mode',
                    'duration_ms', 'time_signature']

evaluation_vector_titles = ['keyTransition', 'tempoTransition', 'trackNumTransition', 'loudnessTransition', 'timbreTransition',
                            'danceabilityTransition', 'energyTransition', 'speechinessTransition','acousticnessTransition',
                            'instrumentalnessTransition', 'livenessTransition', 'valenceTransition']

In [ ]:
# Get all track data (has been run, no longer needed)

# x = 1
# for playlist_id in playlist_ids:
#     print('Running playlist ' + playlist_id)

#     # Get track data
    
#     results = sp.playlist_tracks(playlist_id)
#     tracks = results['items']
#     track_ids = []
#     for track in tracks:
#         track_ids.append(track['track']['id'])

#     columns_to_remove = ['analysis_url', 'type', 'uri', 'track_href']
#     headings = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
#                 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
#                 'id', 'duration_ms', 'time_signature', 'track_title', 'album_title',
#                 'album_artist', 'track_number', 'total_tracks']

#     playlist_data = pd.DataFrame(columns=headings)
#     playlist_length = len(track_ids)

#     i = 0
#     for track in track_ids:
#         results = sp.audio_features(track)
#         if results[0]: 
#             features = results[0]
        
#         features_matrix = pd.DataFrame.from_records(features, index=[0])
        
#         # Remove unneeded columns
#         features_matrix.drop(columns = columns_to_remove, axis = 1, inplace = True)

#         # Add track info
#         features_matrix['track_title'] = tracks[i]['track']['name']
#         features_matrix['album_title'] = tracks[i]['track']['album']['name']
#         features_matrix['album_artist'] = tracks[i]['track']['album']['artists'][0]['name']

#         # Add track number and total tracks
#         features_matrix['track_number'] = tracks[i]['track']['track_number']
#         features_matrix['total_tracks'] = playlist_length

#         playlist_data = pd.concat([playlist_data, features_matrix])
#         i += 1

#     playlist_data = playlist_data.reset_index(drop=True)
#     playlist_data_full = pd.DataFrame(playlist_data)

#     # Get timbre data

#     timbre = pd.DataFrame(columns = ['song_timbre', 'song_timbre_start', 'song_timbre_end', 'loudness_start', 'loudness_end'])

#     for track in track_ids:
#         print('Getting timbre for: '+ track)
#         track_timbre = get_timbre(track, playlist_data_full)
#         timbre = pd.concat([timbre, track_timbre], axis=0)

#     timbre = timbre.reset_index(drop=True)
#     playlist_data = pd.concat([playlist_data, timbre], axis=1)

#     playlist_data.to_csv('./data/test_set/playlist_data_' + str(x) + '.csv')
#     playlist_data_full.to_csv('./data/test_set/playlist_data_full_' + str(x) + '.csv')
#     x += 1

In [ ]:
# # get permissions to rearrange
# from spotipy.oauth2 import SpotifyOAuth
# scope = "playlist-modify-public"
# sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope,redirect_uri='http://localhost:5678/',client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

# # rearrange
# sorted_ids = list(playlist_data_full['id'])

# UPDATED_PLAYLIST = sp.playlist_replace_items(playlist_id,sorted_ids)
# UPDATED_PLAYLIST

In [ ]:
# Evalute all tracks: ExtraTrees, Album Tests

x = 1
sorted_evals = []
original_evals = []
mean_squared_errors = []
eval_changes = []

for playlist_id in playlist_ids:
    
    # print('Running playlist ' + playlist_id)

    # Read and set up data
    playlist_data = pd.read_csv('./data/test_set/playlist_data_' + random_y + str(x) + '.csv')
    playlist_data.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)
    playlist_data_full = playlist_data.copy(deep=True)
    x += 1

    # Remove unnecessary testing columns
    playlist_data.drop(columns = columns_to_remove, axis = 1, inplace = True)

    # Evaluate original order
    original_order_eval = evaluate_data(playlist_data_full)
    original_evals.append(original_order_eval)

    # Load and run trained model
    model = pickle.load(open('./regression/random_forest.sav', 'rb')) # change to whichever model we want to use
    playlist_data_full['order'] = model.predict(playlist_data.values)
    playlist_data_full = playlist_data_full.sort_values(by=['order'])
    playlist_data_full['order'] = np.arange(1, playlist_data_full.shape[0]+1) # convert order to integer playlist track number

    # Evaluate new order
    sorted_order_eval = evaluate_data(playlist_data_full)
    sorted_evals.append(sorted_order_eval)

    y_true = playlist_data_full['track_number']
    y_pred = playlist_data_full['order']
    mean_squared_errors.append(mean_squared_error(y_true, y_pred))

    track_eval_changes = []
    
    for metric in range(len(original_order_eval)):
        track_eval_changes.append(sorted_order_eval[metric] - original_order_eval[metric])
        
    eval_changes.append(track_eval_changes)


In [ ]:
original_evals_df = pd.DataFrame.from_records(original_evals, columns=evaluation_vector_titles)
sorted_evals_df = pd.DataFrame.from_records(sorted_evals, columns=evaluation_vector_titles)
eval_changes_df = pd.DataFrame.from_records(eval_changes, columns=evaluation_vector_titles)

trackNumTransitionOriginalMean = original_evals_df['trackNumTransition'].mean()
trackNumTransitionSortedMean = sorted_evals_df['trackNumTransition'].mean()
trackNumTransitionMeanChange = eval_changes_df['trackNumTransition'].mean()

original_evals_df.drop(columns = ['trackNumTransition', 'loudnessTransition'], axis = 1, inplace = True)
original_evals_df['trackTotal'] = original_evals_df.mean(numeric_only=True, axis=1)
original_evals_df.loc['metricTotalAlbumOriginal'] = original_evals_df.mean(numeric_only=True, axis=0)
featureTransitionOriginalMean = original_evals_df.loc['metricTotalAlbumOriginal','trackTotal']

sorted_evals_df.drop(columns = ['trackNumTransition', 'loudnessTransition'], axis = 1, inplace = True)
sorted_evals_df['trackTotal'] = sorted_evals_df.mean(numeric_only=True, axis=1)
sorted_evals_df.loc['metricTotalAlbumOriginal'] = sorted_evals_df.mean(numeric_only=True, axis=0)
featureTransitionSortedMean = sorted_evals_df.loc['metricTotalAlbumOriginal','trackTotal']

eval_changes_df.drop(columns = ['trackNumTransition', 'loudnessTransition'], axis = 1, inplace = True)
eval_changes_df['trackTotal'] = eval_changes_df.mean(numeric_only=True, axis=1)
eval_changes_df.loc['metricTotalAlbumOriginal'] = eval_changes_df.mean(numeric_only=True, axis=0)
featureTransitionMeanChange = eval_changes_df.loc['metricTotalAlbumOriginal','trackTotal']

In [ ]:
# Evalute all tracks: ExtraTrees with Autoencoder, Album Tests

encoder = tf.keras.models.load_model('./regression/encoder.h5')

x = 1
sorted_evals_auto = []
original_evals_auto = []
mean_squared_errors_auto = []
eval_changes_auto = []

for playlist_id in playlist_ids:
    
    # print('Running playlist ' + playlist_id)

    # Read and set up data
    playlist_data = pd.read_csv('./data/test_set/playlist_data_' + random_y + str(x) + '.csv')
    playlist_data.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)
    playlist_data_full = playlist_data.copy(deep=True)
    x += 1

    # Remove unnecessary testing columns
    playlist_data.drop(columns = columns_to_remove, axis = 1, inplace = True)

    # Evaluate original order
    original_order_eval = evaluate_data(playlist_data_full)
    original_evals_auto.append(original_order_eval)

    # Load and run trained model
    encoded_playlist_data = encoder.predict(playlist_data)
    model = pickle.load(open('./regression/autoencoder_random_forest.sav', 'rb')) # change to whichever model we want to use
    playlist_data_full['order'] = model.predict(encoded_playlist_data)
    playlist_data_full = playlist_data_full.sort_values(by=['order'])
    playlist_data_full['order'] = np.arange(1, playlist_data_full.shape[0]+1) # convert order to integer playlist track number

    # Evaluate new order
    sorted_order_eval = evaluate_data(playlist_data_full)
    sorted_evals_auto.append(sorted_order_eval)

    y_true = playlist_data_full['track_number']
    y_pred = playlist_data_full['order']
    mean_squared_errors_auto.append(mean_squared_error(y_true, y_pred))

    track_eval_changes_auto = []
    
    for metric in range(len(original_order_eval)):
        track_eval_changes_auto.append(sorted_order_eval[metric] - original_order_eval[metric])
        
    eval_changes_auto.append(track_eval_changes_auto)


In [ ]:
original_evals_auto_df = pd.DataFrame.from_records(original_evals_auto, columns=evaluation_vector_titles)
sorted_evals_auto_df = pd.DataFrame.from_records(sorted_evals_auto, columns=evaluation_vector_titles)
eval_changes_auto_df = pd.DataFrame.from_records(eval_changes_auto, columns=evaluation_vector_titles)

trackNumTransitionOriginalMean_auto = original_evals_auto_df['trackNumTransition'].mean()
trackNumTransitionSortedMean_auto = sorted_evals_auto_df['trackNumTransition'].mean()
trackNumTransitionMeanChange_auto = eval_changes_auto_df['trackNumTransition'].mean()

original_evals_auto_df.drop(columns = ['trackNumTransition', 'loudnessTransition'], axis = 1, inplace = True)
original_evals_auto_df['trackTotal'] = original_evals_auto_df.mean(numeric_only=True, axis=1)
original_evals_auto_df.loc['metricTotalAlbumOriginal'] = original_evals_auto_df.mean(numeric_only=True, axis=0)
featureTransitionOriginalMean_auto = original_evals_auto_df.loc['metricTotalAlbumOriginal','trackTotal']

sorted_evals_auto_df.drop(columns = ['trackNumTransition', 'loudnessTransition'], axis = 1, inplace = True)
sorted_evals_auto_df['trackTotal'] = sorted_evals_auto_df.mean(numeric_only=True, axis=1)
sorted_evals_auto_df.loc['metricTotalAlbumOriginal'] = sorted_evals_auto_df.mean(numeric_only=True, axis=0)
featureTransitionSortedMean_auto = sorted_evals_auto_df.loc['metricTotalAlbumOriginal','trackTotal']

eval_changes_auto_df.drop(columns = ['trackNumTransition', 'loudnessTransition'], axis = 1, inplace = True)
eval_changes_auto_df['trackTotal'] = eval_changes_auto_df.mean(numeric_only=True, axis=1)
eval_changes_auto_df.loc['metricTotalAlbumOriginal'] = eval_changes_auto_df.mean(numeric_only=True, axis=0)
featureTransitionMeanChange_auto = eval_changes_auto_df.loc['metricTotalAlbumOriginal','trackTotal']

In [ ]:
# Evalute all tracks: ExtraTrees Predict Next with unoptimised search, Album Tests

x = 1
sorted_evals_next = []
original_evals_next = []
mean_squared_errors = []
eval_changes_next = []

for playlist_id in playlist_ids_random:
    
    # print('Running playlist ' + playlist_id)

    # Read and set up data
    playlist_data = pd.read_csv('./data/test_set/playlist_data_' + random_y + str(x) + '.csv')
    playlist_data.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)
    playlist_data_full = playlist_data.copy(deep=True)
    playlist_data.loc[:,['loudness', 'tempo', 'loudness_start', 'loudness_end']] = MinMaxScaler().fit_transform(playlist_data.loc[:,['loudness', 'tempo', 'loudness_start', 'loudness_end']])
    x += 1

    # Remove unnecessary testing columns
    playlist_data.drop(columns = columns_to_remove, axis = 1, inplace = True)

    # Evaluate original order
    original_order_eval = evaluate_data(playlist_data_full)
    original_evals_next.append(original_order_eval)

    # Load and run trained model
    model = pickle.load(open('./regression/rf_predict_next.sav', 'rb')) # change to whichever model we want to use
    optimal_songs = model.predict(playlist_data.values)
    
    # Order songs based on optimal result
    sorted_order = [0]
    for current_track in range(playlist_data.shape[0]-1):
        distances = []
        for other_track in range(playlist_data.shape[0]):
            # print(other_track)
            if current_track != other_track:
                distances.append(np.linalg.norm(optimal_songs[current_track]-playlist_data.iloc[other_track]))
            else:
                distances.append(1)
        distances_temp = distances.copy()
        # print('D ' + str(distances))
        for other_track in range(len(distances)):
            # print('OT ' + str(other_track))
            # print('D_Temp ' + str(distances_temp))
            if distances.index(min(distances_temp)) not in sorted_order:
                # print('SO ' + str(distances.index(min(distances_temp))))
                sorted_order.append(distances.index(min(distances_temp)))
                break
            else:
                # print('Removed ' + str(min(distances_temp)))
                distances_temp.remove(min(distances_temp))
        # print(str(other_track) + ' ' + str(distances))
        # print('end' + str(distances))
        # sorted()

    playlist_data_full['order'] = sorted_order
    playlist_data_full = playlist_data_full.sort_values(by=['order'])
    playlist_data_full['order'] = np.arange(1, playlist_data_full.shape[0]+1) # convert order to integer playlist track number
        
    # Evaluate new order
    sorted_order_eval = evaluate_data(playlist_data_full)
    sorted_evals_next.append(sorted_order_eval)

    # y_true = playlist_data_full['track_number']
    # y_pred = playlist_data_full['order']
    # mean_squared_errors_random.append(mean_squared_error(y_true, y_pred))

    track_eval_changes = []

    for metric in range(len(original_order_eval)):
        track_eval_changes.append(sorted_order_eval[metric] - original_order_eval[metric])
        
    eval_changes_next.append(track_eval_changes)


In [ ]:
original_evals_next_df = pd.DataFrame.from_records(original_evals_next, columns=evaluation_vector_titles)
sorted_evals_next_df = pd.DataFrame.from_records(sorted_evals_next, columns=evaluation_vector_titles)
eval_changes_next_df = pd.DataFrame.from_records(eval_changes_next, columns=evaluation_vector_titles)

trackNumTransitionOriginalMean_next = original_evals_next_df['trackNumTransition'].mean()
trackNumTransitionSortedMean_next = sorted_evals_next_df['trackNumTransition'].mean()
trackNumTransitionMeanChange_next = eval_changes_next_df['trackNumTransition'].mean()

original_evals_next_df.drop(columns = ['trackNumTransition', 'loudnessTransition'], axis = 1, inplace = True)
original_evals_next_df['trackTotal'] = original_evals_next_df.mean(numeric_only=True, axis=1)
original_evals_next_df.loc['metricTotalAlbumOriginal'] = original_evals_next_df.mean(numeric_only=True, axis=0)
featureTransitionOriginalMean_next = original_evals_next_df.loc['metricTotalAlbumOriginal','trackTotal']

sorted_evals_next_df.drop(columns = ['trackNumTransition', 'loudnessTransition'], axis = 1, inplace = True)
sorted_evals_next_df['trackTotal'] = sorted_evals_next_df.mean(numeric_only=True, axis=1)
sorted_evals_next_df.loc['metricTotalAlbumOriginal'] = sorted_evals_next_df.mean(numeric_only=True, axis=0)
featureTransitionSortedMean_next = sorted_evals_next_df.loc['metricTotalAlbumOriginal','trackTotal']

eval_changes_next_df.drop(columns = ['trackNumTransition', 'loudnessTransition'], axis = 1, inplace = True)
eval_changes_next_df['trackTotal'] = eval_changes_next_df.mean(numeric_only=True, axis=1)
eval_changes_next_df.loc['metricTotalAlbumOriginal'] = eval_changes_next_df.mean(numeric_only=True, axis=0)
featureTransitionMeanChange_next = eval_changes_next_df.loc['metricTotalAlbumOriginal','trackTotal']

In [58]:
# Evalute all tracks: ExtraTrees Predict Next with Optimised Search, Album Tests

x = 1
sorted_evals_search = []
original_evals_search = []
mean_squared_errors = []
eval_changes_search = []

for playlist_id in playlist_ids:
    
    # print('Running playlist ' + playlist_id)

    # Read and set up data
    playlist_data = pd.read_csv('./data/test_set/playlist_data_' + random_y + str(x) + '.csv')
    playlist_data.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)
    playlist_data.drop(index = [6, 7, 8, 9, 10, 11], axis = 0, inplace = True)
    playlist_data_full = playlist_data.copy(deep=True)
    playlist_data.loc[:,['loudness', 'tempo', 'loudness_start', 'loudness_end']] = MinMaxScaler().fit_transform(playlist_data.loc[:,['loudness', 'tempo', 'loudness_start', 'loudness_end']])
    x += 1

    # Remove unnecessary testing columns
    playlist_data.drop(columns = columns_to_remove, axis = 1, inplace = True)

    # Evaluate original order
    original_order_eval = evaluate_data(playlist_data_full)
    original_evals_search.append(original_order_eval)

    # Load and run trained model
    model = pickle.load(open('./regression/rf_predict_next.sav', 'rb')) # change to whichever model we want to use
    optimal_songs = model.predict(playlist_data.values)
    
    # Get all distances between all songs
    all_distances = []
    for current_track in range(playlist_data.shape[0]):
        distances = []
        for other_track in range(playlist_data.shape[0]):
            # print(other_track)
            if current_track != other_track:
                distances.append(np.linalg.norm(optimal_songs[current_track]-playlist_data.iloc[other_track]))
            else:
                distances.append(1000)
        all_distances.append(distances)

    # sorted_order = [0]
    # current_track = 0
    # flag = False
    # while current_track < playlist_data.shape[0]:
    #     if flag == False:
    #         current_track_distances = all_distances[current_track].copy()
    #     if len(current_track_distances) == 0:
    #         break
    #     worst_transition = max(current_track_distances)
    #     while worst_transition > 4:
    #         current_track_distances.remove(worst_transition)
    #         worst_transition = max(current_track_distances)
    #     if all_distances[current_track].index(worst_transition) not in sorted_order:
    #         sorted_order.append(all_distances[current_track].index(worst_transition))
    #         flag = False
    #     else:
    #         flag = True
    #         current_track_distances.remove(worst_transition)
    #         current_track -= 1
    #     current_track += 1

    # print(sorted_order)
    
    # # UCS to find shortest distance through album
    # frontier = []
    # explored = []
    # path_costs = []
    # depth = 0

    # for i in range(1, len(all_distances[0])):
    #     frontier.append([0, all_distances[0].index(all_distances[0][i])])
    #     path_costs.append(all_distances[0][i])
    # depth += 1

    # # Explore
    # while frontier != []:
    # # while min(path_costs) < 1000:
    # # while min(path_costs) < len(all_distances) * 10:
    #     lowest_cost_node = frontier[path_costs.index(min(path_costs))]
    #     lowest_cost_node = [x for x in lowest_cost_node if math.isnan(x) == False]
    #     print(lowest_cost_node, min(path_costs))
    #     depth = len(lowest_cost_node) + 1
    #     explored.append(lowest_cost_node)
    #     frontier.remove(lowest_cost_node)
    #     path_costs.remove(min(path_costs))
    #     if depth < len(all_distances):
    #         for i in range(len(all_distances[depth])):
    #             if all_distances[depth].index(all_distances[depth][i]) not in lowest_cost_node:
    #                 if depth < len(all_distances) - 2:
    #                     if all_distances[depth][i] < 1: # If the distance > 1, it's probably not the right path?
    #                         next_node = lowest_cost_node.copy()
    #                         next_node.append(all_distances[depth].index(all_distances[depth][i]))
    #                         frontier.append(next_node)
    #                         path_costs.append(min(path_costs) + all_distances[depth][i])
    #                 else:
    #                     next_node = lowest_cost_node.copy()
    #                     next_node.append(all_distances[depth].index(all_distances[depth][i]))
    #                     frontier.append(next_node)
    #                     path_costs.append(min(path_costs) + all_distances[depth][i])

    # sorted_order = lowest_cost_node

    playlist_data_full['order'] = sorted_order
    playlist_data_full = playlist_data_full.sort_values(by=['order'])
    playlist_data_full['order'] = np.arange(1, playlist_data_full.shape[0]+1) # convert order to integer playlist track number
        
    # Evaluate new order
    sorted_order_eval = evaluate_data(playlist_data_full)
    sorted_evals_search.append(sorted_order_eval)

    # y_true = playlist_data_full['track_number']
    # y_pred = playlist_data_full['order']
    # mean_squared_errors_random.append(mean_squared_error(y_true, y_pred))

    track_eval_changes = []

    for metric in range(len(original_order_eval)):
        track_eval_changes.append(sorted_order_eval[metric] - original_order_eval[metric])
        
    eval_changes_search.append(track_eval_changes)

[0, 2] 0.667969372777661
[0, 3] 0.7862283608761998
[0, 5] 0.870029999217032
[0, 1] 1.0097800402887007
[0, 3, 2] 1.4859412905313025
[0, 2, 5] 1.6231612622775193
[0, 5, 2] 1.625691331602971
[0, 4] 1.6547985684563773
[0, 3, 5] 1.7069629006183515
[0, 2, 1] 1.7515749536279235
[0, 3, 1] 1.8353765919687555
[0, 5, 1] 1.9751266330404245
[0, 1, 2] 2.101852581845573
[0, 4, 2] 2.322874191932622
[0, 1, 5] 2.3228741919326223
[0, 4, 5] 2.543895802019671
[0, 2, 5, 3] 2.656302078115018
[0, 4, 1] 2.6723094933700753
[0, 5, 2, 3] 2.6854093149684246
[0, 3, 5, 2] 2.6956552407446854
[0, 3, 2, 5] 2.7135983147672764
[0, 3, 2, 1] 2.716910168617538
[0, 2, 5, 1] 2.71944023794299
[0, 5, 2, 1] 2.7485474747963963
[0, 3, 5, 1] 2.8453238599679422
[0, 2, 1, 3] 2.8659873384808026
[0, 3, 1, 2] 2.9192069201571864
[0, 2, 1, 5] 2.9258136444585126
[0, 5, 1, 2] 3.045932868962335
[0, 3, 1, 5] 3.0655636855301815
[0, 5, 1, 3] 3.13246332835762
[0, 1, 2, 3] 3.353484938444669
[0, 4, 2, 3] 3.3534849384446694
[0, 1, 2, 5] 3.413311244

ValueError: Length of values (5) does not match length of index (6)

In [ ]:
original_evals_search_df = pd.DataFrame.from_records(original_evals_search, columns=evaluation_vector_titles)
sorted_evals_search_df = pd.DataFrame.from_records(sorted_evals_search, columns=evaluation_vector_titles)
eval_changes_search_df = pd.DataFrame.from_records(eval_changes_search, columns=evaluation_vector_titles)

trackNumTransitionOriginalMean_search = original_evals_search_df['trackNumTransition'].mean()
trackNumTransitionSortedMean_search = sorted_evals_search_df['trackNumTransition'].mean()
trackNumTransitionMeanChange_search = eval_changes_search_df['trackNumTransition'].mean()

original_evals_search_df.drop(columns = ['trackNumTransition', 'loudnessTransition'], axis = 1, inplace = True)
original_evals_search_df['trackTotal'] = original_evals_search_df.mean(numeric_only=True, axis=1)
original_evals_search_df.loc['metricTotalAlbumOriginal'] = original_evals_search_df.mean(numeric_only=True, axis=0)
featureTransitionOriginalMean_search = original_evals_search_df.loc['metricTotalAlbumOriginal','trackTotal']

sorted_evals_search_df.drop(columns = ['trackNumTransition', 'loudnessTransition'], axis = 1, inplace = True)
sorted_evals_search_df['trackTotal'] = sorted_evals_search_df.mean(numeric_only=True, axis=1)
sorted_evals_search_df.loc['metricTotalAlbumOriginal'] = sorted_evals_search_df.mean(numeric_only=True, axis=0)
featureTransitionSortedMean_search = sorted_evals_search_df.loc['metricTotalAlbumOriginal','trackTotal']

eval_changes_search_df.drop(columns = ['trackNumTransition', 'loudnessTransition'], axis = 1, inplace = True)
eval_changes_search_df['trackTotal'] = eval_changes_search_df.mean(numeric_only=True, axis=1)
eval_changes_search_df.loc['metricTotalAlbumOriginal'] = eval_changes_search_df.mean(numeric_only=True, axis=0)
featureTransitionMeanChange_search = eval_changes_search_df.loc['metricTotalAlbumOriginal','trackTotal']

In [ ]:
evals_df = pd.DataFrame(np.array([[trackNumTransitionMeanChange, featureTransitionMeanChange],
                                [trackNumTransitionMeanChange_auto, featureTransitionMeanChange_auto],
                                [trackNumTransitionMeanChange_next, featureTransitionMeanChange_next],
                                [trackNumTransitionMeanChange_search, featureTransitionMeanChange_search]
                                ]),
                                columns=['Num Sequential Tracks %', 'Mean Feature Change'],
                                index=['ExtraTrees',
                                'ExtraTrees Autoencoder', 
                                'ExtraTrees Predict Next',
                                'ExtraTrees Predict Next Optimised'
                                ])

evals_df